In [9]:
import os
%pwd

'c:\\Users\\LENOVO\\Downloads\\wine_quality\\End_to_End_Wine_Quality\\research'

In [10]:
os.chdir('../')

In [11]:
%pwd

'c:\\Users\\LENOVO\\Downloads\\wine_quality\\End_to_End_Wine_Quality'

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [13]:
from Wine_Quality.constants import *
from Wine_Quality.utils.common import read_yaml,create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
        )

        return data_ingestion_config

In [15]:
import os
import urllib.request as request
from Wine_Quality import logger
from Wine_Quality.utils.common import get_size

In [16]:
import gdown
import os
from pathlib import Path
from Wine_Quality import logger
from Wine_Quality.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        Downloads the file from the source URL to a local path.
        """
        file_exists = os.path.exists(self.config.local_data_file)
        logger.info(f"Checking file existence: {file_exists}")
        
        if not file_exists:
            try:
                logger.info(f"Downloading file from {self.config.source_URL}")
                # Use gdown to download from Google Drive
                gdown.download(self.config.source_URL, self.config.local_data_file, quiet=False)
                logger.info(f"File downloaded: {self.config.local_data_file}")
            except Exception as e:
                logger.error(f"Error downloading file: {e}")
        else:
            file_size = get_size(Path(self.config.local_data_file))
            logger.info(f"File already exists, size: {file_size}")


In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2024-12-28 12:35:38,829: INFO: common: yaml file: C:\Users\LENOVO\Downloads\wine_quality\End_to_End_Wine_Quality\config\config.yaml loaded successfully]
[2024-12-28 12:35:38,876: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-28 12:35:38,881: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-28 12:35:38,886: INFO: common: created directory at: artifacts]
[2024-12-28 12:35:38,887: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-28 12:35:38,917: INFO: 10219733: Checking file existence: True]
[2024-12-28 12:35:38,919: INFO: 10219733: File already exists, size: ~ 99 KB]
